In [ ]:
import pandas as pd
import re

In [ ]:
def auto_haplo_analysis(snp_dir, cul_infos_dir, haplo_out_dir, pheno_infos_dir, hap_pheno_output_dir):
    # 变异位点过滤
    o_d = pd.read_excel(snp_dir)
    o_d["Variation ID"] = o_d["Variation ID"].str.replace(re.compile("\(.+"), "")
    o_d["Variation ID"] = o_d["Variation ID"].str.strip()
    o_d = o_d.T.iloc[:,6:]
    o_d["flag"] = ""
    for i in range(len(o_d)-1):
        col = list(o_d.iloc[i+1, :len(o_d.columns)-1])
        #flag1 = o_d.iloc[i+1, len(o_d.columns)-1]
        if "N" in col:
            o_d.iloc[i+1, len(o_d.columns)-1] = "del"
        elif "DEL" in col:
            o_d.iloc[i+1, len(o_d.columns)-1] = "del"
        elif len(set(col)) != 2:
            o_d.iloc[i+1, len(o_d.columns)-1] = "del"
        else:
            base1 = list(set(col))[0]
            base2 = list(set(col))[1]
            num1 = list(col).count(base1)
            num2 = list(col).count(base2)
            maf = min(num1, num2)/(num1 + num2)
            if maf < 0.15: # 0.15
                o_d.iloc[i+1, len(o_d.columns)-1] = "del"
    o_d.to_csv("C:/Users/chufeng zhao/Desktop/del_no_del.csv")
    o_d = o_d[o_d["flag"] != "del"]
    o_d = o_d.drop(axis=1, columns="flag")
    o_d.to_csv("C:/Users/chufeng zhao/Desktop/del.csv")
    # 得到变异位点的ID
    snp_locs = list(o_d.index[1:])

    # 得到不同单倍型及对应的栽培稻品种名
    hap_dict = {}
    for i in range(len(o_d.columns)):
        hap = "".join(list(o_d.iloc[1:, i]))
        if hap not in hap_dict.keys():
            hap_dict[hap] = [o_d.iloc[0, i]]
        else:
            hap_dict[hap].append(o_d.iloc[0, i])
    hap_g10_dict = {}
    for k, v in hap_dict.items():
        if len(v) >= 10:
            hap_g10_dict[k] = v
    for k, v in hap_g10_dict.items():
        print(k, len(v))

    # 计算每种单倍型下不同亚种水稻的数目
    cultivar_infos = pd.read_excel(cul_infos_dir)
    cultivar_infos["Subpopulation"] = cultivar_infos["Subpopulation"].str.strip()
    cultivar_infos["Cultivar ID"] = cultivar_infos["Cultivar ID"].str.strip()
    cultivar_infos_part = cultivar_infos[["Cultivar ID", "Subpopulation"]]
    sub_name = list(cultivar_infos_part["Subpopulation"].unique())

    cultivar_infos_dict = {}
    for i in range(len(cultivar_infos_part)):
        cultivar_infos_dict[cultivar_infos_part.iloc[i, 0]] = cultivar_infos_part.iloc[i, 1]

    subsp_dict = {}
    for k, v in hap_g10_dict.items():
        subsp_dict[k] = {}
        for i in v:
            sub = cultivar_infos_dict[i]
            if sub not in subsp_dict[k].keys():
                subsp_dict[k][sub] = 1
            else:
                subsp_dict[k][sub] += 1
        last_sub_name = sub_name - subsp_dict[k].keys()
        for j in last_sub_name:
            subsp_dict[k][j] = 0

    # 单倍型与表型关联分析
    phenotype = pd.read_csv(pheno_infos_dir, encoding="UTF-8")

    ph_df = pd.DataFrame()
    for k, v in hap_g10_dict.items():
        inter = phenotype[phenotype["id_name"].apply(lambda x: x in v)]
        inter["flag"] = str(k)
        ph_df = pd.concat(objs = [ph_df, inter], ignore_index = True) 
    ph_df["hap"] = ""
    countor = 0
    for i in ph_df["flag"].value_counts().index:
        countor += 1
        ph_df["hap"][ph_df["flag"] == i] = ("hap" + str(countor))
    ph_df.to_csv(hap_pheno_output_dir,
                 encoding="UTF-8", index=False)
    
        # 保存单倍型分析的结果
    print(subsp_dict)
    with open(haplo_out_dir, "a") as file:
        file.write("\t".join(snp_locs)+"\n")
        counter1 = 0
        for i in ph_df["flag"].value_counts().index:
            counter1 += 1
            file.write("All"+": "+str(len(hap_g10_dict[i]))+"\t")
            file.write("hap" + str(counter1) + "\t")
            file.write(i + "\t")
            for l in sorted(sub_name):
                file.write(l+": "+str(subsp_dict[i][l])+" ")
            file.write("All"+": "+str(len(hap_g10_dict[i]))+"\n")
        file.write(str(hap_dict))

cul_infos_dir = "E:cultivar infos.xlsx" #不用变
pheno_infos_dir = "E:phenos.csv" #不用变

snp_dir1 = "E:all_vars.xlsx" # 每换一个基因就要改一次 注意将excel表启动编辑
haplo_out_dir = "C:单倍型分析初步结果.txt" #  输出单倍型分析表格
hap_pheno_output_dir = "C:hap_related_phe.csv" # 输出根据单倍型的品种分组表

In [ ]:
auto_haplo_analysis(snp_dir = snp_dir1, cul_infos_dir = cul_infos_dir, 
                    haplo_out_dir = haplo_out_dir, pheno_infos_dir = pheno_infos_dir, 
                    hap_pheno_output_dir = hap_pheno_output_dir)